In [35]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
import time
from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import sys
import pprint
max_chrome_driver_path = '/Users/maxlien/Downloads/chromedriver_mac_arm64/chromedriver'

from dotenv import load_dotenv
import os

def config():
    load_dotenv()
  
config()

username = str(os.getenv('MONOPLE_GMAIL'))
password=str(os.getenv('MONOPLE_PASSWORD'))

In [57]:
class StackExchangeScraper:
    def __init__(self, chrome_driver_path, user_agent=None,username=username, password=password):
        self.driver_path = chrome_driver_path
        options = webdriver.ChromeOptions()
        
        # If a custom user agent is provided, set it. Otherwise, use a common one.
        if user_agent:
            options.add_argument(f"user-agent={user_agent}")
        else:
            options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        
        self.driver = webdriver.Chrome(self.driver_path, options=options)
        self.wait =  WebDriverWait(self.driver, 10)

        self.username =username
        self.password = password

    # ... rest of the class methods remain unchanged ...

# Note: Only the initialization method of the class is shown here for brevity.
# The rest of the class methods would remain unchanged from the previous version.

    def _setup_headless_driver(self):
        """
        Sets up a headless Selenium Chrome driver.
        """
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        self.driver = webdriver.Chrome(self.driver_path, options=options)
    def _wait(self,type, name):
        return self.wait.until(
                EC.presence_of_element_located((type, name))
            )
    def _login(self, username, password):
        
        
        login_button_XPATH = '/html/body/div[1]/header[1]/div/nav/ol/li[4]/a'
        login_button = self._wait(By.XPATH, login_button_XPATH)
        self.driver.execute_script("arguments[0].scrollIntoView();", login_button)
        login_button.click()


        
    

        username_box = self._wait(By.XPATH, '//*[@id="email"]')
        username_box.send_keys(username)

        password_box = self._wait(By.XPATH,'//*[@id="password"]' )
        password_box.send_keys(password)


        submit_button =self._wait(By.XPATH, '//*[@id="submit-button"]') 
        self.driver.execute_script("arguments[0].scrollIntoView();", submit_button)
        submit_button.click()




    def search_elements(self, input):
        self.driver.get('https://stackexchange.com')

        self._login(self.username,self.password)

        search_box = self._wait(By.XPATH,'//*[@id="search"]/div/input')
        search_box.send_keys(input)
        search_box.send_keys(Keys.ENTER)

         # Using the new provided XPath to find the desired element
        try:
            element = self._wait(By.XPATH, '//*[@id="content"]/div/div[3]')
        except TimeoutException:
            print("Element not found or page took too long to load")
            return []
        
        return [element]

    def scrape(self, input):
        """
        Main method to scrape StackExchange for a given query and return the texts in a list.
        """
        url = 'https://stackexchange.com'
        #self._setup_headless_driver()
        
        elements = self.search_elements(input)
        texts = [i.text for i in elements if i.text != '']
        time.sleep(10)
        #self.driver.quit()
        return texts

# Note: As before, the Selenium portion cannot be executed in this environment. 
# The code should be run in a local environment with the appropriate browser driver.


In [58]:
qs = StackExchangeScraper(max_chrome_driver_path)
qs.scrape('what is periodic table')

/var/folders/ty/0sfpr_454fxfc8pv3h3387m00000gq/T/ipykernel_891/1719882426.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.driver_path, options=options)


['What is the \'periodic table\' tag for?\nWhat is periodic-table intended for? The tag wiki excerpt is not very helpful as it just describes what the periodic table is. The periodic table is a systematic arrangement of the known chemical …\ndiscussiontags\nasked Sep 16, 2015 at 20:46 on Chemistry Meta Stack Exchange\nWhat is the meaning of the math expression in the periodic table example?\nAs topic Here is the periodic table 1 example : code line 298,299 const phi = Math.acos( - 1 + ( 2 * i ) / l ); const theta = Math.sqrt( l * Math.PI ) * phi; Above is a segment code of periodic table …\nthree.js\nasked Apr 3, 2022 at 7:57 on Stack Overflow\nWhat is the best book or site to understand how to construct Periodic Table, theoretically?\nI want to understand how Periodic table is constructed theoretically and mathematically. But I can\'t find self-contained book or site about it by myself since I\'m a computer science guy and lack of k …\nquantum-mechanicsresource-recommendationsatomic-p